### SQL Alchemy is Python's ORM (Object Relational Mapping) Library

[Reference #1](https://docs.sqlalchemy.org/en/13/core/metadata.html?highlight=sqlalchemy%20sql%20schema%20table#sqlalchemy.schema.Table)
[Reference #2](https://www.pythonsheets.com/notes/python-sqlalchemy.html)

In [1]:
from sqlalchemy import create_engine
from sqlalchemy import inspect
from sqlalchemy import MetaData
from sqlalchemy import select
from sqlalchemy import Table
from sqlalchemy.orm import sessionmaker
import os
import pprint

### Working with Existing Database

In [2]:
username = os.environ['windowsuser']
pwd = os.environ['windowspwd']
server = 'VCLO49529.am.mds.honda.com'
port = '5432'
database = 'techtribes'

In [3]:
engine = create_engine(f'postgresql://{username}:{pwd}@{server}:{port}/{database}')

In [4]:
inspector = inspect(engine)

In [5]:
print(inspector.get_table_names())

['basic_member_info', 'lab_visitors_log', 'think_tank_metrics']


### Create metadata object

In [6]:
metadata = MetaData()
metadata.reflect(bind=engine)
pprint.pprint(metadata.tables)

{'basic_member_info': Table('basic_member_info', MetaData(bind=None), Column('first_name', VARCHAR(length=20), table=<basic_member_info>), Column('last_name', VARCHAR(length=20), table=<basic_member_info>), Column('hondaweb_id', VARCHAR(length=20), table=<basic_member_info>, primary_key=True, nullable=False), Column('hondaweb_url', VARCHAR(length=250), table=<basic_member_info>), Column('is_ahm', VARCHAR(length=3), table=<basic_member_info>), schema=None),
 'lab_visitors_log': Table('lab_visitors_log', MetaData(bind=None), Column('When', TIMESTAMP(), table=<lab_visitors_log>), Column('Location', TEXT(), table=<lab_visitors_log>), Column('Who_Came_For_Help', TEXT(), table=<lab_visitors_log>), Column('Who_Helped', TEXT(), table=<lab_visitors_log>), Column('What', TEXT(), table=<lab_visitors_log>), Column('Resolution', TEXT(), table=<lab_visitors_log>), Column('Tags', TEXT(), table=<lab_visitors_log>), schema=None),
 'think_tank_metrics': Table('think_tank_metrics', MetaData(bind=None), C

### Create table object

In [7]:
lab_log = metadata.tables['lab_visitors_log']

In [8]:
lab_log.columns.keys()

['When',
 'Location',
 'Who_Came_For_Help',
 'Who_Helped',
 'What',
 'Resolution',
 'Tags']

### Create a session and then query

[Reference #1](https://docs.sqlalchemy.org/en/13/core/metadata.html?highlight=sqlalchemy%20sql%20schema%20table#sqlalchemy.schema.Table)
[Reference #2](https://www.pythonsheets.com/notes/python-sqlalchemy.html)

In [9]:
Session = sessionmaker(bind=engine)
session = Session()

#### Can query using ```filter_by():```

In [10]:
query = session.query(lab_log).filter_by(Who_Helped='Dan Dafler')

#### Can also query using ```filter():```

In [30]:
session.query(lab_log).filter(lab_log.columns.Who_Helped.like('%Dan%')).first()

(datetime.datetime(2019, 1, 18, 0, 0),
 'MMC',
 'Ryan Smith',
 'Dan Dafler, Han Zou, Lyndsay Rush, Miguel Miranda, Ryan Smith',
 'Monthly report works, runs slower now than implementation; many external links',
 None,
 'Pandas')

In [33]:
for row in session.query(lab_log).filter(lab_log.columns.Who_Helped.like('%Dan%')).all():
    print(row)

(datetime.datetime(2019, 1, 18, 0, 0), 'MMC', 'Ryan Smith', 'Dan Dafler, Han Zou, Lyndsay Rush, Miguel Miranda, Ryan Smith', 'Monthly report works, runs slower now than implementation; many external links', None, 'Pandas')
(datetime.datetime(2019, 1, 18, 0, 0), 'MMC', 'Bob Keating', 'Dan Dafler, Han Zou, Lyndsay Rush, Miguel Miranda, Ryan Smith', 'Losing It team tracking sheet', 'UserForms', 'Excel VBA')
(datetime.datetime(2019, 3, 29, 0, 0), 'MMC', 'Catherine Doran', 'Lyndsay Rush, Dan Dafler', 'Excel radar chart formatting', None, 'Excel')
(datetime.datetime(2019, 3, 29, 0, 0), 'MMC', 'Han Zou', 'Lyndsay Rush,\xa0Dan Dafler', 'How to set up RSA - not available in HondaWeb', None, 'ITSM')
(datetime.datetime(2019, 5, 3, 0, 0), 'MMC', 'Catherine Doran', 'Dan Dafler, Ryan Smith', 'Real-time data for team to use in supplier discussions', None, 'Business Objects')
(datetime.datetime(2019, 5, 10, 0, 0), 'MMC', 'Catherine Doran, Ken Winkle', 'Dan Dafler, Ryan Smith, Miguel Miranda', 'Real-ti

### DELETE / Empty a table

In [ ]:
with engine.connect() as conn:
    del_stmt = lab_log.delete()
    conn.execute(del_stmt)

### DROP a table

In [ ]:
lab_log.drop(engine)